In [1]:

# TEST: BAYESIAN STACK IMPORTS

import pymc as pm
import arviz as az
import pandas as pd
import numpy as np

print("✅ PyMC + ArviZ + Pandas + NumPy imported successfully")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
C:\Users\HP\anaconda3\envs\amr_bayes\Lib\site-packages\arviz\__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


✅ PyMC + ArviZ + Pandas + NumPy imported successfully


In [7]:
import os
os.getcwd()  # This is the correct way to call the getcwd function from the os module

'C:\\Users\\HP\\anaconda_projects\\8afe27f8-b8f5-4108-af11-a3d510e87fb0'

In [8]:
# LOAD DATASET (LOCAL FOLDER VERSION)

df = pd.read_csv("antibiotic_resistance_tracking (Project Data).csv")

print("Dataset Loaded Successfully.")
print("Shape of dataset:", df.shape)

df.head()

Dataset Loaded Successfully.
Shape of dataset: (2200, 16)


,Patient_ID,Age,Gender,Specimen_Type,Amoxicillin,Ciprofloxacin,Meropenem,Vancomycin,Colistin,Test_Method,Resistance_Genes,Outcome,Age Group,Resistance Score,MDR Status,Gene Detected
0,P0001,36,Female,Blood,Intermediate,Sensitive,Intermediate,Intermediate,Intermediate,Automated System,KPC,ICU,Adult,0,No,Yes
1,P0002,59,Female,Blood,Intermediate,Resistant,Intermediate,Sensitive,Intermediate,MIC,KPC,Recovered,Adult,1,No,Yes
2,P0003,13,Female,Urine,Intermediate,Sensitive,Resistant,Intermediate,Resistant,MIC,OXA-48,ICU,Teenager,2,No,Yes
3,P0004,4,Female,Sputum,Intermediate,Resistant,Sensitive,Intermediate,Intermediate,Automated System,OXA-48,ICU,Child,1,No,Yes
4,P0005,51,Male,Urine,Resistant,Sensitive,Resistant,Intermediate,Intermediate,Automated System,NaN,ICU,Adult,2,No,No


In [9]:
# 1) CHECK COLUMNS AND BASIC INFO

print("Number of rows, columns:", df.shape)
print("\nCOLUMNS:\n", list(df.columns))

print("\nINFO:")
df.info()

Number of rows, columns: (2200, 16)

COLUMNS:
 ['Patient_ID', 'Age', 'Gender', 'Specimen_Type', 'Amoxicillin', 'Ciprofloxacin', 'Meropenem', 'Vancomycin', 'Colistin', 'Test_Method', 'Resistance_Genes', 'Outcome', 'Age Group', 'Resistance Score', 'MDR Status', 'Gene Detected']

INFO:
<class 'pandas.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Patient_ID        2199 non-null   str  
 1   Age               2200 non-null   int64
 2   Gender            2200 non-null   str  
 3   Specimen_Type     2200 non-null   str  
 4   Amoxicillin       2200 non-null   str  
 5   Ciprofloxacin     2200 non-null   str  
 6   Meropenem         2200 non-null   str  
 7   Vancomycin        2200 non-null   str  
 8   Colistin          2200 non-null   str  
 9   Test_Method       2200 non-null   str  
 10  Resistance_Genes  1763 non-null   str  
 11  Outcome           2200 non-nul

In [10]:
# 2) CHECK UNIQUE VALUES FOR KEY FIELDS
key_cols = [
    "Gender", "Specimen_Type", "Test_Method", "Outcome",
    "Age Group", "MDR Status", "Gene Detected"
]

for col in key_cols:
    print("\n" + "="*60)
    print("COLUMN:", col)
    print("Unique values:", df[col].dropna().unique()[:20])  # show first 20 unique values
    print("Missing values:", df[col].isna().sum())


COLUMN: Gender
Unique values: <StringArray>
['Female', 'Male']
Length: 2, dtype: str
Missing values: 0

COLUMN: Specimen_Type
Unique values: <StringArray>
['Blood', 'Urine', 'Sputum', 'Wound swab', 'Stool']
Length: 5, dtype: str
Missing values: 0

COLUMN: Test_Method
Unique values: <StringArray>
['Automated System', 'MIC', 'Disc Diffusion']
Length: 3, dtype: str
Missing values: 0

COLUMN: Outcome
Unique values: <StringArray>
['ICU', 'Recovered', 'Deceased']
Length: 3, dtype: str
Missing values: 0

COLUMN: Age Group
Unique values: <StringArray>
['Adult', 'Teenager', 'Child', 'Senior']
Length: 4, dtype: str
Missing values: 0

COLUMN: MDR Status
Unique values: <StringArray>
['No', 'Yes']
Length: 2, dtype: str
Missing values: 0

COLUMN: Gene Detected
Unique values: <StringArray>
['Yes', 'No']
Length: 2, dtype: str
Missing values: 0


In [11]:
# CHECK UNIQUE VALUES FOR ANTIBIOTIC RESULTS

antibiotics = [
    "Amoxicillin",
    "Ciprofloxacin",
    "Meropenem",
    "Vancomycin",
    "Colistin"
]

for ab in antibiotics:
    print("\n" + "="*60)
    print("ANTIBIOTIC:", ab)
    print("Unique values:", df[ab].dropna().unique())
    print("Missing values:", df[ab].isna().sum())


ANTIBIOTIC: Amoxicillin
Unique values: <StringArray>
['Intermediate', 'Resistant', 'Sensitive']
Length: 3, dtype: str
Missing values: 0

ANTIBIOTIC: Ciprofloxacin
Unique values: <StringArray>
['Sensitive', 'Resistant', 'Intermediate']
Length: 3, dtype: str
Missing values: 0

ANTIBIOTIC: Meropenem
Unique values: <StringArray>
['Intermediate', 'Resistant', 'Sensitive']
Length: 3, dtype: str
Missing values: 0

ANTIBIOTIC: Vancomycin
Unique values: <StringArray>
['Intermediate', 'Sensitive', 'Resistant']
Length: 3, dtype: str
Missing values: 0

ANTIBIOTIC: Colistin
Unique values: <StringArray>
['Intermediate', 'Resistant', 'Sensitive']
Length: 3, dtype: str
Missing values: 0


In [12]:
# RECODE ANTIBIOTIC RESULTS INTO BINARY RESISTANCE
# Resistant + Intermediate = 1
# Sensitive = 0

map_resistance = {
    "Resistant": 1,
    "Intermediate": 1,
    "Sensitive": 0
}

antibiotics = [
    "Amoxicillin",
    "Ciprofloxacin",
    "Meropenem",
    "Vancomycin",
    "Colistin"
]

for ab in antibiotics:
    df[ab + "_bin"] = df[ab].map(map_resistance)

print("Binary columns created successfully.")
df[[ab + "_bin" for ab in antibiotics]].head()

Binary columns created successfully.


,Amoxicillin_bin,Ciprofloxacin_bin,Meropenem_bin,Vancomycin_bin,Colistin_bin
0,1,0,1,1,1
1,1,1,1,0,1
2,1,0,1,1,1
3,1,1,0,1,1
4,1,0,1,1,1


In [13]:
# VERIFY BINARY OUTCOMES

for ab in antibiotics:
    print("\n", ab + "_bin")
    print("Unique values:", df[ab + "_bin"].unique())
    print("Missing values:", df[ab + "_bin"].isna().sum())


 Amoxicillin_bin
Unique values: [1 0]
Missing values: 0

 Ciprofloxacin_bin
Unique values: [0 1]
Missing values: 0

 Meropenem_bin
Unique values: [1 0]
Missing values: 0

 Vancomycin_bin
Unique values: [1 0]
Missing values: 0

 Colistin_bin
Unique values: [1 0]
Missing values: 0


In [14]:
# PREPARE DATA FOR CIPROFLOXACIN MODEL

# Select outcome
ab = "Ciprofloxacin_bin"

# Copy dataset (good practice)
d = df.copy()

# Create binary predictors
d["MDR_bin"] = (d["MDR Status"] == "Yes").astype(int)
d["Gene_bin"] = (d["Gene Detected"] == "Yes").astype(int)

# Create category codes for hierarchical grouping
d["specimen_code"] = d["Specimen_Type"].astype("category").cat.codes
d["agegroup_code"] = d["Age Group"].astype("category").cat.codes

# Store category levels (important for interpretation later)
specimen_levels = d["Specimen_Type"].astype("category").cat.categories
agegroup_levels = d["Age Group"].astype("category").cat.categories

print("Data prepared successfully.")
print("Number of observations:", len(d))
print("Number of specimen groups:", len(specimen_levels))
print("Number of age groups:", len(agegroup_levels))

Data prepared successfully.
Number of observations: 2200
Number of specimen groups: 5
Number of age groups: 4


In [ ]:
# BAYESIAN HIERARCHICAL LOGISTIC MODEL (CIPROFLOXACIN)
# Outcome: Ciprofloxacin_bin (1 = Resistant/Intermediate, 0 = Sensitive)
# Predictors: MDR_bin, Gene_bin
# Random intercepts: Specimen_Type, Age Group

y = d["Ciprofloxacin_bin"].values.astype(int)
mdr = d["MDR_bin"].values
gene = d["Gene_bin"].values
spec_idx = d["specimen_code"].values
age_idx = d["agegroup_code"].values

n_spec = len(specimen_levels)
n_age = len(agegroup_levels)

coords = {
    "obs_id": np.arange(len(d)),
    "specimen": specimen_levels,
    "agegroup": agegroup_levels
}

with pm.Model(coords=coords) as cip_model:
    # ----------------------------
    # Priors for fixed effects
    # ----------------------------
    alpha = pm.Normal("alpha", mu=0, sigma=2.5)
    beta_mdr = pm.Normal("beta_mdr", mu=0, sigma=2.5)
    beta_gene = pm.Normal("beta_gene", mu=0, sigma=2.5)

    # ----------------------------
    # Random effects (non-centered)
    # ----------------------------
    sigma_spec = pm.HalfCauchy("sigma_spec", beta=2.5)
    sigma_age = pm.HalfCauchy("sigma_age", beta=2.5)

    z_spec = pm.Normal("z_spec", mu=0, sigma=1, dims="specimen")
    z_age = pm.Normal("z_age", mu=0, sigma=1, dims="agegroup")

    u_spec = pm.Deterministic("u_spec", z_spec * sigma_spec, dims="specimen")
    u_age = pm.Deterministic("u_age", z_age * sigma_age, dims="agegroup")

    # ----------------------------
    # Linear predictor (logit link)
    # ----------------------------
    eta = alpha + beta_mdr * mdr + beta_gene * gene + u_spec[spec_idx] + u_age[age_idx]
    p = pm.Deterministic("p", pm.math.sigmoid(eta), dims="obs_id")

    # Likelihood
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y, dims="obs_id")

    # Sampling
    cip_idata = pm.sample(
        draws=1500,
        tune=1500,
        chains=4,
        target_accept=0.9,
        random_seed=42
    )

In [ ]:

# CHECK WHETHER THE MODEL FINISHED AND cip_idata EXISTS

"cip_idata" in globals()

In [16]:

# KERNEL HEALTH CHECK

1 + 1

2

In [17]:

# IMPORT LIBRARIES (AFTER RESTART)


import pymc as pm
import arviz as az
import pandas as pd
import numpy as np

print("Libraries loaded successfully.")

Libraries loaded successfully.


In [18]:
# Load dataset from current notebook folder
df = pd.read_csv("antibiotic_resistance_tracking (Project Data).csv")

print("Shape:", df.shape)
df.head()

Shape: (2200, 16)


,Patient_ID,Age,Gender,Specimen_Type,Amoxicillin,Ciprofloxacin,Meropenem,Vancomycin,Colistin,Test_Method,Resistance_Genes,Outcome,Age Group,Resistance Score,MDR Status,Gene Detected
0,P0001,36,Female,Blood,Intermediate,Sensitive,Intermediate,Intermediate,Intermediate,Automated System,KPC,ICU,Adult,0,No,Yes
1,P0002,59,Female,Blood,Intermediate,Resistant,Intermediate,Sensitive,Intermediate,MIC,KPC,Recovered,Adult,1,No,Yes
2,P0003,13,Female,Urine,Intermediate,Sensitive,Resistant,Intermediate,Resistant,MIC,OXA-48,ICU,Teenager,2,No,Yes
3,P0004,4,Female,Sputum,Intermediate,Resistant,Sensitive,Intermediate,Intermediate,Automated System,OXA-48,ICU,Child,1,No,Yes
4,P0005,51,Male,Urine,Resistant,Sensitive,Resistant,Intermediate,Intermediate,Automated System,NaN,ICU,Adult,2,No,No


In [19]:
# Recode antibiotic outcomes: Resistant/Intermediate=1, Sensitive=0
map_resistance = {"Resistant": 1, "Intermediate": 1, "Sensitive": 0}

antibiotics = ["Amoxicillin", "Ciprofloxacin", "Meropenem", "Vancomycin", "Colistin"]

for ab in antibiotics:
    df[ab + "_bin"] = df[ab].map(map_resistance)

print(df[[ab + "_bin" for ab in antibiotics]].nunique())

Amoxicillin_bin      2
Ciprofloxacin_bin    2
Meropenem_bin        2
Vancomycin_bin       2
Colistin_bin         2
dtype: int64


In [20]:
# Prepare data for Ciprofloxacin hierarchical model

ab = "Ciprofloxacin_bin"

d = df.copy()

# Binary predictors
d["MDR_bin"] = (d["MDR Status"] == "Yes").astype(int)
d["Gene_bin"] = (d["Gene Detected"] == "Yes").astype(int)

# Encode grouping variables
d["specimen_code"] = d["Specimen_Type"].astype("category").cat.codes
d["agegroup_code"] = d["Age Group"].astype("category").cat.codes

# Store levels for later interpretation
specimen_levels = d["Specimen_Type"].astype("category").cat.categories
agegroup_levels = d["Age Group"].astype("category").cat.categories

print("Observations:", len(d))
print("Specimen groups:", len(specimen_levels))
print("Age groups:", len(agegroup_levels))

Observations: 2200
Specimen groups: 5
Age groups: 4


In [2]:
# Reload dataset after kernel restart
import pandas as pd

df = pd.read_csv("antibiotic_resistance_tracking (Project Data).csv")

print("Dataset loaded:", df.shape)

Dataset loaded: (2200, 16)


In [3]:
# Recreate binary antibiotic columns
map_resistance = {"Resistant": 1, "Intermediate": 1, "Sensitive": 0}
antibiotics = ["Amoxicillin", "Ciprofloxacin", "Meropenem", "Vancomycin", "Colistin"]

for ab in antibiotics:
    df[ab + "_bin"] = df[ab].map(map_resistance)

print(df[[ab + "_bin" for ab in antibiotics]].nunique())

Amoxicillin_bin      2
Ciprofloxacin_bin    2
Meropenem_bin        2
Vancomycin_bin       2
Colistin_bin         2
dtype: int64


In [4]:
# Prepare data for Ciprofloxacin model
d = df.copy()

d["MDR_bin"] = (d["MDR Status"] == "Yes").astype(int)
d["Gene_bin"] = (d["Gene Detected"] == "Yes").astype(int)

d["specimen_code"] = d["Specimen_Type"].astype("category").cat.codes
d["agegroup_code"] = d["Age Group"].astype("category").cat.codes

specimen_levels = d["Specimen_Type"].astype("category").cat.categories
agegroup_levels = d["Age Group"].astype("category").cat.categories

print("Prepared:", len(d), "rows,", len(specimen_levels), "specimen groups,", len(agegroup_levels), "age groups")

Prepared: 2200 rows, 5 specimen groups, 4 age groups


In [ ]:
import numpy as np
import pymc as pm

y = d["Ciprofloxacin_bin"].values.astype(int)
mdr = d["MDR_bin"].values
gene = d["Gene_bin"].values
spec_idx = d["specimen_code"].values
age_idx = d["agegroup_code"].values

coords = {
    "obs_id": np.arange(len(d)),
    "specimen": specimen_levels,
    "agegroup": agegroup_levels
}

with pm.Model(coords=coords) as cip_model:
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    sigma_spec = pm.HalfCauchy("sigma_spec", 2.5)
    sigma_age = pm.HalfCauchy("sigma_age", 2.5)

    z_spec = pm.Normal("z_spec", 0, 1, dims="specimen")
    z_age = pm.Normal("z_age", 0, 1, dims="agegroup")

    u_spec = pm.Deterministic("u_spec", z_spec * sigma_spec, dims="specimen")
    u_age = pm.Deterministic("u_age", z_age * sigma_age, dims="agegroup")

    eta = alpha + beta_mdr * mdr + beta_gene * gene + u_spec[spec_idx] + u_age[age_idx]
    p = pm.math.sigmoid(eta)

    pm.Bernoulli("y_obs", p=p, observed=y, dims="obs_id")

    cip_idata = pm.sample(
        draws=400,
        tune=400,
        chains=2,
        cores=1,
        target_accept=0.9,
        random_seed=42,
        progressbar=False
    )

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
C:\Users\HP\anaconda3\envs\amr_bayes\Lib\site-packages\arviz\__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha, beta_mdr, beta_gene, sigma_spec, sigma_age, z_spec, z_age]


In [ ]:
import pymc as pm
import numpy as np

y = df["Ciprofloxacin_bin"].values
mdr = (df["MDR Status"] == "Yes").astype(int).values
gene = (df["Gene Detected"] == "Yes").astype(int).values

with pm.Model() as simple_model:
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    eta = alpha + beta_mdr * mdr + beta_gene * gene
    p = pm.math.sigmoid(eta)

    pm.Bernoulli("y_obs", p=p, observed=y)

    simple_idata = pm.sample(
        draws=300,
        tune=300,
        chains=2,
        cores=1,
        progressbar=False,
        random_seed=42
    )

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha, beta_mdr, beta_gene]


In [ ]:
import pymc as pm
import numpy as np
import pandas as pd

df = pd.read_csv("antibiotic_resistance_tracking (Project Data).csv")

map_resistance = {"Resistant": 1, "Intermediate": 1, "Sensitive": 0}
df["Ciprofloxacin_bin"] = df["Ciprofloxacin"].map(map_resistance)

y = df["Ciprofloxacin_bin"].values
mdr = (df["MDR Status"] == "Yes").astype(int).values
gene = (df["Gene Detected"] == "Yes").astype(int).values

with pm.Model() as speed_model:
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    eta = alpha + beta_mdr * mdr + beta_gene * gene
    p = pm.math.sigmoid(eta)

    pm.Bernoulli("y_obs", p=p, observed=y)

    speed_idata = pm.sample(
        draws=300,
        tune=300,
        chains=2,
        cores=1,
        progressbar=False,
        random_seed=42
    )

In [1]:
import os

# Force PyTensor to use the C++ compiler (g++) and compile in fast mode
os.environ["PYTENSOR_FLAGS"] = "cxx=g++,mode=FAST_RUN,optimizer=fast_compile"

In [2]:
import pytensor
print("PYTENSOR_FLAGS =", pytensor.config.cxx, pytensor.config.mode)

PYTENSOR_FLAGS = g++ FAST_RUN


In [3]:
import pymc as pm
import numpy as np
import pandas as pd

df = pd.read_csv("antibiotic_resistance_tracking (Project Data).csv")
map_resistance = {"Resistant": 1, "Intermediate": 1, "Sensitive": 0}
df["Ciprofloxacin_bin"] = df["Ciprofloxacin"].map(map_resistance)

y = df["Ciprofloxacin_bin"].values
mdr = (df["MDR Status"] == "Yes").astype(int).values
gene = (df["Gene Detected"] == "Yes").astype(int).values

with pm.Model() as test_model:
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    p = pm.math.sigmoid(alpha + beta_mdr * mdr + beta_gene * gene)
    pm.Bernoulli("y_obs", p=p, observed=y)

    test_idata = pm.sample(
        draws=100,
        tune=100,
        chains=2,
        cores=1,
        random_seed=42
    )

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha, beta_mdr, beta_gene]


C:\Users\HP\anaconda3\envs\amr_bayes\Lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 100 tune and 100 draw iterations (200 + 200 draws total) took 4 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [4]:
with pm.Model() as proper_model:
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    p = pm.math.sigmoid(alpha + beta_mdr * mdr + beta_gene * gene)
    pm.Bernoulli("y_obs", p=p, observed=y)

    proper_idata = pm.sample(
        draws=1000,
        tune=1000,
        chains=4,
        cores=1,
        target_accept=0.9,
        random_seed=42
    )

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [alpha, beta_mdr, beta_gene]


C:\Users\HP\anaconda3\envs\amr_bayes\Lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 69 seconds.


In [5]:
import arviz as az
az.summary(proper_idata, round_to=2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.38,0.10,0.19,0.57,0.0,0.0,1512.03,1701.87,1.0
beta_mdr,1.05,0.13,0.80,1.32,0.0,0.0,2383.31,2162.14,1.0
beta_gene,0.12,0.11,-0.10,0.32,0.0,0.0,1575.23,1947.61,1.0


In [7]:
# Create clean integer indices
specimen_idx = df["Specimen_Type"].astype("category").cat.codes.to_numpy(dtype="int32")
age_idx = df["Age Group"].astype("category").cat.codes.to_numpy(dtype="int32")

specimen_levels = df["Specimen_Type"].astype("category").cat.categories
age_levels = df["Age Group"].astype("category").cat.categories

print(specimen_idx.dtype, age_idx.dtype, specimen_idx.min(), specimen_idx.max(), age_idx.min(), age_idx.max())

int32 int32 0 4 0 3


In [8]:
# Fit hierarchical model using pm.Data + dims

coords = {"specimen": specimen_levels, "agegroup": age_levels}

with pm.Model(coords=coords) as hierarchical_model:

    # Data containers
    mdr_data = pm.Data("mdr", mdr)
    gene_data = pm.Data("gene", gene)
    spec_i = pm.Data("spec_i", specimen_idx)
    age_i = pm.Data("age_i", age_idx)

    # Fixed effects
    alpha = pm.Normal("alpha", 0, 2.5)
    beta_mdr = pm.Normal("beta_mdr", 0, 2.5)
    beta_gene = pm.Normal("beta_gene", 0, 2.5)

    # Random effects (non-centered is more stable)
    sigma_spec = pm.HalfNormal("sigma_spec", 1.0)
    sigma_age = pm.HalfNormal("sigma_age", 1.0)

    z_spec = pm.Normal("z_spec", 0, 1, dims="specimen")
    z_age = pm.Normal("z_age", 0, 1, dims="agegroup")

    u_spec = pm.Deterministic("u_spec", z_spec * sigma_spec, dims="specimen")
    u_age = pm.Deterministic("u_age", z_age * sigma_age, dims="agegroup")

    # Linear predictor
    eta = alpha + beta_mdr * mdr_data + beta_gene * gene_data + u_spec[spec_i] + u_age[age_i]
    p = pm.math.sigmoid(eta)

    pm.Bernoulli("y_obs", p=p, observed=y)

    hierarchical_idata = pm.sample(
        draws=1000,
        tune=1000,
        chains=4,
        cores=1,
        target_accept=0.9,
        random_seed=42
    )

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [alpha, beta_mdr, beta_gene, sigma_spec, sigma_age, z_spec, z_age]


C:\Users\HP\anaconda3\envs\amr_bayes\Lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 377 seconds.
There were 5 divergences after tuning. Increase `target_accept` or reparameterize.


In [9]:
import arviz as az
az.summary(hierarchical_idata, var_names=["alpha","beta_mdr","beta_gene","sigma_spec","sigma_age"], round_to=2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.37,0.15,0.10,0.67,0.0,0.00,1400.20,1350.50,1.0
beta_mdr,1.05,0.13,0.82,1.32,0.0,0.00,2988.79,2128.67,1.0
beta_gene,0.12,0.12,-0.09,0.34,0.0,0.00,2600.53,2501.61,1.0
sigma_spec,0.12,0.12,0.00,0.32,0.0,0.01,1053.32,1472.13,1.0
sigma_age,0.12,0.12,0.00,0.32,0.0,0.00,1179.94,1677.59,1.0


In [10]:
target_accept=0.95